In [1]:
import pickle
import pandas as pd
import implicit
import lightfm
import scipy

import string
# Библиотека построения индекса приближенного поиска ближайших соседей
import annoy
import numpy as np

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split

c:\study\venv\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [5]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

import re
def preprocess_txt(line):
    line = re.sub(r'\.|\"|\,', ' ', line)
    line = re.sub('\sх\s', 'x', line)
    line = re.sub('\"', ' ', line)
    line = re.sub('\-', ' ', line)

    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [i for i in spls if not i.isdigit()]

    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]

    return spls



 ## Задача
Разработать рекомендательную систему для аптек. Представьте, что вы приходите на кассу, а у кассира во время формирования чека всплывает подсказка, что можно вам ещё порекомендовать.

Для этого рассмотрим датасеты - преобразуем соответствующие колонки, удалим пропуски и прочее

Сначала рассмотрим датасет со списоком типов продуктов. Предположим, что первый элемент массива это идентификатора клиента. 

In [6]:
with open('./data/Product_dict.pkl', 'rb') as f:
    dataTest = pickle.load(f)
dataTest = pd.DataFrame.from_dict(dataTest,  orient='index')

In [7]:
dataTest

,0
168308,(197312) Пакет-майка 25см х 45см 906
134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
168570,(197309) Пакет 28см х 50см 906
146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"
...,...
193603,(110044169) название -1
193484,(110050486) название -1
192459,(110020427) название -1
193510,(110047818) название -1


In [8]:
dataTest.isnull().values.any()

False

In [9]:
dataTest = pd.DataFrame.from_dict(dataTest)
dataTest['user_id'] = dataTest.index
dataTest['item_id'] = dataTest[[0]]
dataTest['user_id'] = dataTest['user_id'].astype(int)

dataTest = dataTest.reset_index()
dataTest = dataTest[['user_id', 'item_id']]
dataTest

,user_id,item_id
0,168308,(197312) Пакет-майка 25см х 45см 906
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
3,168570,(197309) Пакет 28см х 50см 906
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"
...,...,...
30413,193603,(110044169) название -1
30414,193484,(110050486) название -1
30415,192459,(110020427) название -1
30416,193510,(110047818) название -1


Как показано у описания товара есть идентификтор и само описание выделим отдельно эти две сущности. Также видим что у некторых товаров отсутствует название, в этом случае удалим такие товары. Так при если нет названия то мы не можем просто клиенту предложить такой товар

In [10]:
def preprocessTxt(line, filter = True):
    line = line.lower()
    lines = line.split(')')
    
    if len(lines) >= 2 and filter:
        line = lines[1].strip()
        return line
    
    if len(lines) >= 2 and not filter:
        line = lines[0].replace("(", "").strip()
        return line

dataTest['itemName'] = dataTest["item_id"].apply(preprocessTxt, args=(True,))
dataTest['itemId'] = dataTest["item_id"].apply(preprocessTxt, args=(False,))

In [11]:
dataTest

,user_id,item_id,itemName,itemId
0,168308,(197312) Пакет-майка 25см х 45см 906,пакет-майка 25см х 45см 906,197312
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...,перекись водорода р-р наружн. 3% фл.полимерн. ...,62448
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...,салициловая кислота р-р спирт 2% фл 40мл n1 404,72183
3,168570,(197309) Пакет 28см х 50см 906,пакет 28см х 50см 906,197309
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906","пакет ""аптека озерки"" 28 х 35см 906",111023
...,...,...,...,...
30413,193603,(110044169) название -1,название -1,110044169
30414,193484,(110050486) название -1,название -1,110050486
30415,192459,(110020427) название -1,название -1,110020427
30416,193510,(110047818) название -1,название -1,110047818


In [174]:
dataTest = dataTest[['user_id', 'itemName', 'itemId']]
dataTest['user_id'] = dataTest['user_id'].astype(int)
dataTest['itemId'] = dataTest['itemId'].astype(str)
dataTest

<ipython-input-174-63e398d6b6d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTest['user_id'] = dataTest['user_id'].astype(int)
<ipython-input-174-63e398d6b6d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTest['itemId'] = dataTest['itemId'].astype(str)


,user_id,itemName,itemId
0,168308,пакет-майка 25см х 45см 906,197312
1,134832,перекись водорода р-р наружн. 3% фл.полимерн. ...,62448
2,101384,салициловая кислота р-р спирт 2% фл 40мл n1 404,72183
3,168570,пакет 28см х 50см 906,197309
4,146960,"пакет ""аптека озерки"" 28 х 35см 906",111023
...,...,...,...
30413,193603,название -1,110044169
30414,193484,название -1,110050486
30415,192459,название -1,110020427
30416,193510,название -1,110047818


In [175]:
dataGood = dataTest[(dataTest['itemName'] != "название -1") ]
dataGood

,user_id,itemName,itemId
0,168308,пакет-майка 25см х 45см 906,197312
1,134832,перекись водорода р-р наружн. 3% фл.полимерн. ...,62448
2,101384,салициловая кислота р-р спирт 2% фл 40мл n1 404,72183
3,168570,пакет 28см х 50см 906,197309
4,146960,"пакет ""аптека озерки"" 28 х 35см 906",111023
...,...,...,...
30401,153665,очки корригирующие кемнер оптикс глянцевые пла...,121891
30402,164193,лактулоза форте пор.15мл №10 597,190054
30403,153013,курапрокс набор (зубная паста вайт из блэк 90м...,121465
30404,59719,лиерак премиум набор бархат новогодний (крем 5...,117974


In [176]:
dataGood.to_csv('./data/goods.csv', index=False)

In [177]:
dataGood['text'] = dataGood['itemName'].apply(lambda x: preprocess_txt(str(x)))
dataGood

<ipython-input-177-13d19c7dc279>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataGood['text'] = dataGood['itemName'].apply(lambda x: preprocess_txt(str(x)))


,user_id,itemName,itemId,text
0,168308,пакет-майка 25см х 45см 906,197312,"[пакет, майк, 25смx45см]"
1,134832,перекись водорода р-р наружн. 3% фл.полимерн. ...,62448,"[перекись, водород, р, р, наружн, фл, полимерн..."
2,101384,салициловая кислота р-р спирт 2% фл 40мл n1 404,72183,"[салициловый, кислота, р, р, спирт, фл, 40мл, n1]"
3,168570,пакет 28см х 50см 906,197309,"[пакет, 28смx50см]"
4,146960,"пакет ""аптека озерки"" 28 х 35см 906",111023,"[пакет, аптека, озерко, 28x35см]"
...,...,...,...,...
30401,153665,очки корригирующие кемнер оптикс глянцевые пла...,121891,"[очки, корригировать, кемнер, оптикс, глянцевы..."
30402,164193,лактулоза форте пор.15мл №10 597,190054,"[лактулоза, форт, 15мл, №10]"
30403,153013,курапрокс набор (зубная паста вайт из блэк 90м...,121465,"[курапрокс, набор, зубной, паста, вайта, блэк,..."
30404,59719,лиерак премиум набор бархат новогодний (крем 5...,117974,"[лиерак, премиум, набор, бархат, новогодний, к..."


# Обработка датасета покупок
Дынные датасета чековых покупок. Удалим пропуски данных  и преобразуем данные как прошлой порцией данных

In [12]:
checkData = pd.read_csv('./data/chackData.csv')

c:\study\venv\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
checkData.dropna(inplace=True)
checkData

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"
...,...,...,...,...,...,...,...,...,...
19999995,2018-06-13,1601618,1499.0,66842.0,(111992) Кэа Хэлс Ромашка [цветки фильтр-пакет...,615.0,140.0,1838.0,"1,00"
19999996,2018-06-13,1394104,1495.0,136795.0,(97857) Лориста Н таб. п.о. 50мг+12.5мг №90,738.0,170.0,-1.0,"1,00"
19999997,2018-06-13,1570654,1516.0,119513.0,(25299) Локрен тб п/о 20мг N28,738.0,170.0,-1.0,"1,00"
19999998,2018-06-13,1924036,1485.0,71723.0,(60907) Тералиджен табл. п.п.о. 5 мг №25,637.0,146.0,-1.0,"1,00"


In [14]:
checkData['itemName'] = checkData["name"].apply(preprocessTxt, args=(True,))
checkData['itemId'] = checkData["name"].apply(preprocessTxt, args=(False,))

In [15]:
checkData['Data'] = pd.to_datetime(checkData['sale_date_date'], format='%Y-%m-%d')
checkData.rename({'contact_id': 'user_id'}, axis=1, inplace=True)
checkData

,sale_date_date,user_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity,itemName,itemId,Data
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00",пакет-майка 25см х 45см,197312,2018-12-07
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00",перекись водорода р-р наружн. 3% фл.полимерн. ...,62448,2018-12-07
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00",салициловая кислота р-р спирт 2% фл 40мл n1,72183,2018-12-07
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00",пакет 28см х 50см,197309,2018-12-07
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00",пакет 30см х 60см,197310,2018-12-07
...,...,...,...,...,...,...,...,...,...,...,...,...
19999995,2018-06-13,1601618,1499.0,66842.0,(111992) Кэа Хэлс Ромашка [цветки фильтр-пакет...,615.0,140.0,1838.0,"1,00",кэа хэлс ромашка [цветки фильтр-пакеты 1г] n20,111992,2018-06-13
19999996,2018-06-13,1394104,1495.0,136795.0,(97857) Лориста Н таб. п.о. 50мг+12.5мг №90,738.0,170.0,-1.0,"1,00",лориста н таб. п.о. 50мг+12.5мг №90,97857,2018-06-13
19999997,2018-06-13,1570654,1516.0,119513.0,(25299) Локрен тб п/о 20мг N28,738.0,170.0,-1.0,"1,00",локрен тб п/о 20мг n28,25299,2018-06-13
19999998,2018-06-13,1924036,1485.0,71723.0,(60907) Тералиджен табл. п.п.о. 5 мг №25,637.0,146.0,-1.0,"1,00",тералиджен табл. п.п.о. 5 мг №25,60907,2018-06-13


In [16]:
dataTransform = checkData[['Data','user_id','product_id', 'itemId','itemName', 'quantity']].copy()
dataTransform.reset_index()
dataTransform

,Data,user_id,product_id,itemId,itemName,quantity
0,2018-12-07,1260627,168308.0,197312,пакет-майка 25см х 45см,"1,00"
1,2018-12-07,198287,134832.0,62448,перекись водорода р-р наружн. 3% фл.полимерн. ...,"1,00"
2,2018-12-07,2418385,101384.0,72183,салициловая кислота р-р спирт 2% фл 40мл n1,"1,00"
3,2018-12-07,1285774,168570.0,197309,пакет 28см х 50см,"1,00"
4,2018-12-07,1810323,168319.0,197310,пакет 30см х 60см,"1,00"
...,...,...,...,...,...,...
19999995,2018-06-13,1601618,66842.0,111992,кэа хэлс ромашка [цветки фильтр-пакеты 1г] n20,"1,00"
19999996,2018-06-13,1394104,136795.0,97857,лориста н таб. п.о. 50мг+12.5мг №90,"1,00"
19999997,2018-06-13,1570654,119513.0,25299,локрен тб п/о 20мг n28,"1,00"
19999998,2018-06-13,1924036,71723.0,60907,тералиджен табл. п.п.о. 5 мг №25,"1,00"


In [17]:
dataTransform['user_id'] = dataTransform['user_id'].astype(int)
dataTransform['product_id'] = dataTransform['product_id'].astype(int)
dataTransform['quantity'] = dataTransform['quantity'].astype(str).str.replace(',', '.').astype(float)

dataTransform['quantity'] = dataTransform.apply(lambda row: 1 if (row["quantity"] > 0) else 0, axis=1)
dataTransform

,Data,user_id,product_id,itemId,itemName,quantity
0,2018-12-07,1260627,168308,197312,пакет-майка 25см х 45см,1
1,2018-12-07,198287,134832,62448,перекись водорода р-р наружн. 3% фл.полимерн. ...,1
2,2018-12-07,2418385,101384,72183,салициловая кислота р-р спирт 2% фл 40мл n1,1
3,2018-12-07,1285774,168570,197309,пакет 28см х 50см,1
4,2018-12-07,1810323,168319,197310,пакет 30см х 60см,1
...,...,...,...,...,...,...
19999995,2018-06-13,1601618,66842,111992,кэа хэлс ромашка [цветки фильтр-пакеты 1г] n20,1
19999996,2018-06-13,1394104,136795,97857,лориста н таб. п.о. 50мг+12.5мг №90,1
19999997,2018-06-13,1570654,119513,25299,локрен тб п/о 20мг n28,1
19999998,2018-06-13,1924036,71723,60907,тералиджен табл. п.п.о. 5 мг №25,1


In [185]:
dataTransform.to_csv('./data/checkBuy.csv', index=False)

In [18]:
dataTransform.head(5)

,Data,user_id,product_id,itemId,itemName,quantity
0,2018-12-07,1260627,168308,197312,пакет-майка 25см х 45см,1
1,2018-12-07,198287,134832,62448,перекись водорода р-р наружн. 3% фл.полимерн. ...,1
2,2018-12-07,2418385,101384,72183,салициловая кислота р-р спирт 2% фл 40мл n1,1
3,2018-12-07,1285774,168570,197309,пакет 28см х 50см,1
4,2018-12-07,1810323,168319,197310,пакет 30см х 60см,1


In [19]:
dataTransform[dataTransform['user_id'] == 2520366]

,Data,user_id,product_id,itemId,itemName,quantity
2337288,2018-11-21,2520366,158549,181234,pl шприц одноразовый инсулиновый 1мл №1,1
2337799,2018-11-21,2520366,158586,181229,pl шприц одноразовый 3-комп. 2мл №1,1
2338402,2018-11-21,2520366,158600,181232,pl шприц одноразовый 3-комп. 5мл №1,1
2387394,2018-11-21,2520366,162832,185573,минки ватные диски №100,1
2485383,2018-11-21,2520366,22832,57208,"пакет нд ""майка"" 28х50",1
2485933,2018-11-21,2520366,22470,114118,pl вода питьевая негазированная 500мл,1
2487993,2018-11-21,2520366,51228,107758,pl аскорбинка с сахаром таб. №10,1


Валидационный датасет. Разумнее всего сделать его из датасета checkData, по последней дате покупки для каждого пользователя. Выберем по последней дате 

In [197]:
validateData = dataTransform.sort_values(by=['Data','user_id'] ,ascending=True ).drop_duplicates(['user_id'], keep="last")[[ "Data" , "user_id", "itemId"]]
validateData
# validateData.to_csv('./data/validatdataTransformeData.csv', index=False)


,Data,user_id,itemId
4743900,2018-01-01,399,61227
4744412,2018-01-01,12207,51910
4744392,2018-01-01,17512,65438
4743837,2018-01-01,20239,55073
6685757,2018-01-01,22182,58542
...,...,...,...
115256,2018-12-09,2742143,120201
115342,2018-12-09,2743330,184313
235812,2018-12-09,2746142,89316
115269,2018-12-09,2746260,109962


удалим товары ндс - как ненужный элемент

In [20]:
dataTransform = dataTransform[(dataTransform['itemId'] != '48791') ]
dataTransform.reset_index()
dataTransform

,Data,user_id,product_id,itemId,itemName,quantity
0,2018-12-07,1260627,168308,197312,пакет-майка 25см х 45см,1
1,2018-12-07,198287,134832,62448,перекись водорода р-р наружн. 3% фл.полимерн. ...,1
2,2018-12-07,2418385,101384,72183,салициловая кислота р-р спирт 2% фл 40мл n1,1
3,2018-12-07,1285774,168570,197309,пакет 28см х 50см,1
4,2018-12-07,1810323,168319,197310,пакет 30см х 60см,1
...,...,...,...,...,...,...
19999995,2018-06-13,1601618,66842,111992,кэа хэлс ромашка [цветки фильтр-пакеты 1г] n20,1
19999996,2018-06-13,1394104,136795,97857,лориста н таб. п.о. 50мг+12.5мг №90,1
19999997,2018-06-13,1570654,119513,25299,локрен тб п/о 20мг n28,1
19999998,2018-06-13,1924036,71723,60907,тералиджен табл. п.п.о. 5 мг №25,1


После проверки оказалось, что клиенты редко когда повторяються в течении всего промежутка времени. Обычно для одного клиента есть одна покупка - больше информации нет, тем более мы можем прикрутить к системе рекомендации еще один кандидатогенератор, который будет рандомно предлагать товары из ранее купленных. 

Можно предложить валидационный датасет, тогда когда система будет работать в боевом режиме  И у нас будет информация что из предложенного клиент купил. А в настоящей время простой повтор ранее купленных товаров будет работать лучше, чем сложные алгоритмы

Но в целях изучения я реализую для них все методы 

 # Факторизационные машины 
 В настоящей релизации нужна огромный массив - показывает ошибку
 Unstacked DataFrame is too big, causing int32 overflow
 Поэтому реализацию таким методом мы отпустим

In [100]:
# shell = pd.pivot_table(
#     dataTransform, 
#     index="user_id", 
#     columns="product_id", 
#     values="quantity"
# )
# shell.head()

# Колаборативннная фильтрация

данные метод применяется для рекоммендательной системе имеющих данные рейтинга
Данная система хороша для оценки фильмы/музыки так как прослушаешь ты ее один раз. Лекарства имеют свойства заканчиваться поэтому проще предлагать те ассоритметы , которые уже  клиентом покупались! 

Можно ввести бинарный признак покупки/непокупки однако для оценки нужно пройстись по всем товарам - а это порядка 36000 асортиментов Это долго - можно оценить купит он продукт или не купит по топу продаваемых товаров 
Но у нас такого топа нет

Поэтому реализуем три кандидатагенератора 
 1. По близким названиям - FastTest
 2. По ранее купленным таварам клиента 
 3. по 1000 самым продаваемым товарам
 4. Предложим пакет обязательно! 

 1000 самых покумаемых товаров

In [21]:
dataTransform = dataTransform[['user_id', 'itemId', 'itemName']]
dataTransform

,user_id,itemId,itemName
0,1260627,197312,пакет-майка 25см х 45см
1,198287,62448,перекись водорода р-р наружн. 3% фл.полимерн. ...
2,2418385,72183,салициловая кислота р-р спирт 2% фл 40мл n1
3,1285774,197309,пакет 28см х 50см
4,1810323,197310,пакет 30см х 60см
...,...,...,...
19999995,1601618,111992,кэа хэлс ромашка [цветки фильтр-пакеты 1г] n20
19999996,1394104,97857,лориста н таб. п.о. 50мг+12.5мг №90
19999997,1570654,25299,локрен тб п/о 20мг n28
19999998,1924036,60907,тералиджен табл. п.п.о. 5 мг №25


In [22]:
# popular = analisData['itemId'].value_counts()
popular = dataTransform['itemId'].value_counts().rename_axis('itemId').reset_index(name='counts')

In [23]:
mostPopular = popular.iloc[:1000]
mostPopular

,itemId,counts
0,197312,118106
1,181542,89851
2,57733,87792
3,117825,77153
4,81509,73543
...,...,...
995,37757,4416
996,28587,4415
997,79902,4403
998,45406,4402


In [228]:
# mostPopular.to_csv('./data/mostPopular.csv', index=False)
# dataTransform.to_csv('./data/dataTransform.csv', index=False)

In [233]:
import random as rnd

mostPopularItem = list(mostPopular['itemId'].values)
users = list(dataTransform['user_id'].values)

usersMostPopular = pd.DataFrame()
for user in users:
    userPopular = dataTransform[(dataTransform['user_id'] == user) &
                                (dataTransform['itemId'].isin(mostPopularItem))]


    if not userPopular.empty:
        userPopular['Buy'] = 1
        usersMostPopular = pd.concat([usersMostPopular, userPopular], ignore_index=True, axis=0).copy()
    else:
        number = rnd.randrange(0, len(mostPopularItem) - 1)
        itemId = mostPopularItem[number]
        itemName = dataTransform[dataTransform['itemId'] == itemId].iloc[0]['itemName']
        userNotBy = {"user_id": [user],
                     'itemId': [mostPopularItem[number]],
                     'itemName': [itemName],
                     'Buy': [0]}
        userPopular = pd.DataFrame.from_dict(userNotBy)
        usersMostPopular = pd.concat([usersMostPopular, userPopular], ignore_index=True, axis=0).copy()

<ipython-input-233-3fbb952a556e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  userPopular['Buy'] = 1


MemoryError: Unable to allocate 18.9 MiB for an array with shape (19820071,) and data type bool

In [24]:
# usersMostPopular.to_csv('./data/usersMostPopular.csv', index=False)
usersMostPopular = pd.read_csv('./data/usersMostPopular.csv')
usersMostPopular

,user_id,itemId,itemName,Buy
0,1260627,197312,пакет-майка 25см х 45см,1
1,1260627,197309,пакет 28см х 50см,1
2,1260627,110048492,название,1
3,1260627,35743,шприцы одноразовые с иглой 5мл n1,1
4,1260627,13804,л-тироксин 50 берлин хеми табл. 50 мкг №50,1
...,...,...,...,...
922705,1069360,60747,вестибо таб. 24 мг. №30,1
922706,1069360,98434,тромбо асс тб п/о кишечнораств 50мг n28,1
922707,1069360,60747,вестибо таб. 24 мг. №30,1
922708,1069360,41942,эгилок тб 50мг n60,1


In [28]:
usersMostPopular.rename({'itemName': 'title','user_id': 'userId', 'Buy': 'rating', }, axis=1, inplace=True)
usersMostPopular = usersMostPopular[['title', 'userId', 'rating']]
usersMostPopular.head(5)

,title,userId,rating
0,пакет-майка 25см х 45см,1260627,1
1,пакет 28см х 50см,1260627,1
2,название,1260627,1
3,шприцы одноразовые с иглой 5мл n1,1260627,1
4,л-тироксин 50 берлин хеми табл. 50 мкг №50,1260627,1


In [247]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [249]:
reader = Reader(rating_scale=(0.0, 1.0))
data = Dataset.load_from_df(usersMostPopular, reader)

In [250]:
trainset, testset = train_test_split(data, test_size=0.15)

algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

test_pred = algo.test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [251]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.0384


0.0384341480939196

In [25]:
import pickle

def saveModel(filname = "", model = None):
    pickle.dump(model, open(filname, 'wb'))
    
def readModel(filname = ""):
    loaded_model = pickle.load(open(filname, 'rb'))    
    return loaded_model

In [257]:
filenameColab = './model/modelColad.txt'
saveModel(filname = filenameColab, model = algo)

Как видно результат не очень хороший - много пользователей и много наименвоаний товаров

Но все равно реализуем в качестве учебы функцию выбора купленного бы товара


In [29]:
titlePopular = list(usersMostPopular['title'].values)
usersMostPopular

,title,userId,rating
0,пакет-майка 25см х 45см,1260627,1
1,пакет 28см х 50см,1260627,1
2,название,1260627,1
3,шприцы одноразовые с иглой 5мл n1,1260627,1
4,л-тироксин 50 берлин хеми табл. 50 мкг №50,1260627,1
...,...,...,...
922705,вестибо таб. 24 мг. №30,1069360,1
922706,тромбо асс тб п/о кишечнораств 50мг n28,1069360,1
922707,вестибо таб. 24 мг. №30,1069360,1
922708,эгилок тб 50мг n60,1069360,1


In [32]:
import random as rnd
filenameColab = './model/modelColad.txt'
modelcolab = readModel(filname = filenameColab)
def generatorUser(userID = None):
    collect = []
    titleSelect = rnd.choice(titlePopular)
    titleProb = modelcolab.predict(uid=userID, iid=titleSelect).est
    while len(collect) < 3:        
        titleSelect = rnd.choice(titlePopular)
        titleProb = modelcolab.predict(uid=userID, iid=titleSelect).est
        if titleProb >= 0.995:
            collect.append(titleSelect)
    return collect 

In [33]:
generatorUser(userID =1260627 )

['ортофен тб п/о 25мг n20',
 'глюкофаж таб.п.п.о.500мг №60',
 'лекролин капли глазн 20мг/мл фл-капел 10мл n1']

## Построены рекомендации с Word2Vec
Псотроим модель Word2Vec и 

In [34]:
dataTransform

,user_id,itemId,itemName
0,1260627,197312,пакет-майка 25см х 45см
1,198287,62448,перекись водорода р-р наружн. 3% фл.полимерн. ...
2,2418385,72183,салициловая кислота р-р спирт 2% фл 40мл n1
3,1285774,197309,пакет 28см х 50см
4,1810323,197310,пакет 30см х 60см
...,...,...,...
19999995,1601618,111992,кэа хэлс ромашка [цветки фильтр-пакеты 1г] n20
19999996,1394104,97857,лориста н таб. п.о. 50мг+12.5мг №90
19999997,1570654,25299,локрен тб п/о 20мг n28
19999998,1924036,60907,тералиджен табл. п.п.о. 5 мг №25


In [35]:
dataTransform['text'] = dataTransform['itemName'].apply(lambda x: preprocess_txt(str(x)))

<ipython-input-35-727f2b1d69f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTransform['text'] = dataTransform['itemName'].apply(lambda x: preprocess_txt(str(x)))


In [36]:
from gensim.models import FastText
from gensim.models import Word2Vec


In [292]:
sentences = [i for i in dataTransform['text'] if len(i) > 2]
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)
modelWV = Word2Vec(sentences=sentences, vector_size=20, min_count=1, window=5)

In [293]:
# modelFT.save("./model/modelFT")
# modelWV.save("./model/modelWV")

In [37]:
modelWV = Word2Vec.load('./model/modelWV')
dataTransform.head(5)

,user_id,itemId,itemName,text
0,1260627,197312,пакет-майка 25см х 45см,"[пакет, майк, 25смx45см]"
1,198287,62448,перекись водорода р-р наружн. 3% фл.полимерн. ...,"[перекись, водород, р, р, наружн, фл, полимерн..."
2,2418385,72183,салициловая кислота р-р спирт 2% фл 40мл n1,"[салициловый, кислота, р, р, спирт, фл, 40мл, n1]"
3,1285774,197309,пакет 28см х 50см,"[пакет, 28смx50см]"
4,1810323,197310,пакет 30см х 60см,"[пакет, 30смx60см]"


In [38]:
modelWV = Word2Vec.load('./model/modelWV')
indexProduct = annoy.AnnoyIndex(20 ,'angular')

index_product = {}
index_itemId = {}
counter = 0

for index, row in dataTransform.iterrows():
    n_w2v = 0
    itemName = row['itemName']
    itemId = row['itemId']
    
    index_product[counter] = itemName
    index_itemId[counter] = itemId

    text = row['text']

    vector = np.zeros(20)
    for word in text:
        if word in modelWV.wv:
            vector += modelWV.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    indexProduct.add_item(counter, vector)
    counter += 1
    
indexProduct.build(10)
indexProduct.save('./model/similarProduct.ann')  

True

In [40]:
index_product

{0: 'пакет-майка 25см х 45см',
 1: 'перекись водорода р-р наружн. 3% фл.полимерн. 100мл',
 2: 'салициловая кислота р-р спирт 2% фл 40мл n1',
 3: 'пакет 28см х 50см',
 4: 'пакет 30см х 60см',
 5: 'кеппра таб.п.п.о.500мг №30',
 6: 'пакет 28см х 50см',
 7: 'пакет-майка 25см х 45см',
 8: 'перекись водорода р-р наружн. 3% фл.полимерн. 100мл',
 9: 'пакет-майка 25см х 45см',
 10: 'пакет "аптека озерки" 28 х 35см',
 11: 'пакет 28см х 50см',
 12: 'карта забота о здоровье',
 13: 'пакет-майка 25см х 45см',
 14: 'пакет-майка 25см х 45см',
 15: 'пакет "аптека озерки" 28 х 35см',
 16: 'пакет-майка 25см х 45см',
 17: 'ранитидин тб 150мг уп n20',
 18: 'пакет-майка 25см х 45см',
 19: 'платки носовые "зева кидс" №10',
 20: 'пакет майка "благодарим за покупку" (28+14',
 21: 'иглы микро-файн д/шприц-ручки 30g 0,3*8мм n100',
 22: 'пакет майка "благодарим за покупку" (28+14',
 23: 'дорзопт капли глазные 2% 5мл. фл.-кап. n3',
 24: 'пакет 28см х 50см',
 25: 'pl шприц одноразовый 3-комп. 3мл №1',
 26: 'pl шпри

In [127]:
modelWV = Word2Vec.load('./model/modelWV')
threshold = 0.02
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(20)
    for word in preprocessed_question:
        if word in modelWV.wv:
            vector += modelWV.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    
    answer_index = indexProduct.get_nns_by_vector(vector, 5, include_distances=True)
    
    
    asnwer = {}
    
    index = answer_index[0]
    distance = answer_index[1]
    for index, item in enumerate(index):
        productName = index_product[item]
        productId = index_itemId[item]
            
        asnwer[productId] = productName

    return asnwer

In [55]:
find_answer('одноразовый')

([2334568, 16524126, 2341561, 2845148, 2923780], [0.7231814861297607, 0.7410405874252319, 0.7609788775444031, 0.7609788775444031, 0.7609788775444031])


{'192163': 'шприц одноразовый 10мл 3-компонентный с иглой 0,8х40мм №1',
 '38826': 'шприцы одноразовые с иглой 0,8х40мм 20мл уп n1',
 '192162': 'шприц одноразовый 5мл 3-компонентный с иглой 0,7х40мм №1'}

### Реализуем функцию выбора пакета


In [99]:
dataPacket = pd.DataFrame()
dataAll = pd.DataFrame()
def getrandomPasket():
    dataPacket = dataTransform[(dataTransform['itemName'].str.startswith("пакет-майка"))]
    
    ItemIgnore = set(list(dataPacket['itemId'].values))
    dataPacket.drop_duplicates(['itemId'], keep='first', inplace=True)    
    
    dataAll = dataTransform[~dataTransform['itemId'].isin(list(ItemIgnore))]
    
    return  dataPacket , dataAll

    

In [100]:
dataPacket , dataAll = getrandomPasket()

c:\study\venv\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [101]:
dataPacket

,user_id,itemId,itemName,text
0,1260627,197312,пакет-майка 25см х 45см,"[пакет, майк, 25смx45см]"
4777167,1786235,200394,пакет-майка 40см х 70см,"[пакет, майк, 40смx70см]"


Логика рекомнедации такая 

 1 Если есть есть номер клиента - предложим ему из того что он раньше покупил (2 товара)
   Предложим ему из 1000 самых популярны товаров
     
 2. Если есть название - предлождим товера по названию
 
 3. В конце предложить пакет



In [136]:
#  функция преложение пользователю того что он покупал ранее
def userOldBuy(userId = None):
    userBuy = dataTransform[dataTransform['user_id'] == int(userId)].copy()
    popularUser = userBuy['itemId'].value_counts().rename_axis('itemId').reset_index(name='counts')
    userItems = rnd.choices(list(popularUser['itemId']), k=3)
    popular = dataTransform[dataTransform['itemId'].isin(userItems)].drop_duplicates(['itemId'], keep='first')

    userOldBuy = {}
    for index, row in popular.iterrows():
        userOldBuy[row['itemId']] = row['itemName']
    return userOldBuy
# функция предожения самог опопулярного товаров
def userPopular(userId =None):
    Popular = generatorUser(userID =userId )

    usersPopular = {}
    for item in Popular:
        posible = dataTransform[dataTransform['itemName'] == item]
        if not posible.empty:
            usersPopular[posible.iloc[0]['itemId']] = posible.iloc[0]['itemName']
    return usersPopular

# функция выбора пакета
def packerChoose():
    paketsDict = {}
    pakets = rnd.choices((dataPacket['itemId'].values), k=1)
    paketsChoose =  dataPacket[dataPacket['itemId'] == pakets[0]]

    itemId = paketsChoose.iloc[0]['itemId']
    itemName = paketsChoose.iloc[0]['itemName'] 
    paketsDict[itemId] = itemName
    return paketsDict
#  выбор по запросу
def requestuser(Request = ''): 
    answers = find_answer('одноразовый')

    requestDict = {}
    for index, item in answers.items():
        userSeg  = dataTransform[dataTransform['itemName'] == item].drop_duplicates(['itemId'], keep='first')
    
        if not userSeg.empty:
            itemId = userSeg.iloc[0]['itemId']
            itemName = userSeg.iloc[0]['itemName'] 
            answers[itemId] = itemName
    return answers    
    

In [148]:
def recomendUser(userId = None, request = None):
    segeust = packerChoose()
    if userId != None:
        userBuy = userOldBuy(userId)
        userPop = userPopular(userId)        
        segeust.update(userBuy)
        segeust.update(userPop)
        
    if request != None:
        requestUser = requestuser(request)
        segeust.update(requestUser)
    return segeust
   
        
    

In [149]:
recomendUser(userId = 1786235, request= 'одноразовый')

{'200394': 'пакет-майка 40см х 70см',
 '62968': 'перекись водорода р-р наружн. 3% фл.полимерн. 100мл',
 '45112': 'присыпка детская 40г уп n1',
 '70458': 'пенталгин таб.п.п.о.№24 (без кодеина',
 '118125': 'кардиомагнил таб.п.п.о.75мг №100',
 '60684': 'тенотен детский таб. №40',
 '192163': 'шприц одноразовый 10мл 3-компонентный с иглой 0,8х40мм №1',
 '38826': 'шприцы одноразовые с иглой 0,8х40мм 20мл уп n1',
 '192162': 'шприц одноразовый 5мл 3-компонентный с иглой 0,7х40мм №1'}

Вывод:

    была рассмотрена системе рекомендации - применена колаборативная фильтрация для наиболее популярных товаров 
    а также построена модель Word2Vec для поиска похожих по названию товаров
    
Вадидацинный сет применять как уже писал ненужна - нет информации о покупке преложенных товаров а сами пользоватедли 
представлены в модели в единичном экзепмпляре (была одна покупка - по одному значению распределение не построишь!)

Из слоожного  - уж больно долго было обрабатывать датасет из почти 20 млн строк  - у меня два раза ломался 
ноутбук (приходилось все перстараивать а это в примерно 7-8 часов)
